In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# pip install lxml

## Import Libraries

In [3]:
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [4]:
# Define Base Url
base_url = "https://www.nfl.com"

# Define Week
current_week = 1

### Get Pages to All Tabs

In [5]:
# Request raw HTML
passing1 = requests.get("https://www.nfl.com/stats/player-stats/category/passing/2023/reg/all/passingyards/desc")

# Create a BeautifulSoup object called `soup` to traverse the combined HTML
soup = BeautifulSoup(passing1.content, "html.parser")

# Find all <a> elements within <li> elements
a_elements = soup.find_all('li', class_='d3-o-tabs__list-item')

# Extract the text from the <a> elements
tabs = [element.find('a').get_text() for element in a_elements]

# Extract the links (href attributes) from the <a> elements
links = [base_url + element.find('a')['href'] for element in a_elements]

# Create a dictionary to store the category names and their corresponding links
stat_pages = dict(zip(tabs, links))

# Print the dictionary as a table-like format using Pandas
stat_pages_df = pd.DataFrame(list(stat_pages.items()), columns=['Category', 'Link'])

print(stat_pages_df)

           Category                                               Link
0           Passing  https://www.nfl.com/stats/player-stats/categor...
1           Rushing  https://www.nfl.com/stats/player-stats/categor...
2         Receiving  https://www.nfl.com/stats/player-stats/categor...
3           Fumbles  https://www.nfl.com/stats/player-stats/categor...
4           Tackles  https://www.nfl.com/stats/player-stats/categor...
5     Interceptions  https://www.nfl.com/stats/player-stats/categor...
6       Field Goals  https://www.nfl.com/stats/player-stats/categor...
7          Kickoffs  https://www.nfl.com/stats/player-stats/categor...
8   Kickoff Returns  https://www.nfl.com/stats/player-stats/categor...
9           Punting  https://www.nfl.com/stats/player-stats/categor...
10     Punt Returns  https://www.nfl.com/stats/player-stats/categor...


### Get All Pages Connected to Each Tab

In [6]:
import requests
from bs4 import BeautifulSoup

base_url = "https://www.nfl.com"

# Initialize the current_link with the starting URL
current_link = "https://www.nfl.com/stats/player-stats/category/rushing/2023/reg/all/rushingyards/desc"

while current_link:
    # Request raw HTML for the current page
    response = requests.get(current_link)

    # Check if the request was successful
    if response.status_code == 200:
        # Create a BeautifulSoup object to parse the HTML
        soup = BeautifulSoup(response.content, "html.parser")

        # Scrape the data from the current page here
        # (You can add your scraping logic here)

        # Find the "Next Page" link
        next_page_link = soup.find('a', class_='nfl-o-table-pagination__next')

        if next_page_link:
            # Extract the 'href' attribute
            href = next_page_link['href']

            # Update current_link with the next page's URL
            current_link = base_url + href
            print("Scraping next page:", current_link)
        else:
            # No more pages to scrape, exit the loop
            print("No more pages to scrape.")
            break
    else:
        print("Error: Unable to fetch the page.")
        break


Scraping next page: https://www.nfl.com/stats/player-stats/category/rushing/2023/REG/all/rushingyards/DESC?aftercursor=AAAAGQAAABdARoAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STBOU0lzSWpNeU1EQTFOelE1TFRSak16SXRPREkyT1MwMFlURTVMV1E0TlRGaFl6azVZbUV6WXlJc0lqSXdNak1pWFgwPQ==
Scraping next page: https://www.nfl.com/stats/player-stats/category/rushing/2023/REG/all/rushingyards/DESC?aftercursor=AAAAMgAAADBANgAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlNaUlzSWpNeU1EQTFORFF4TFRVNU5qRXRNRFUxTXkwNU1UZGpMVE5oTlRabE5XRTBZekkyWXlJc0lqSXdNak1pWFgwPQ==
Scraping next page: https://www.nfl.com/stats/player-stats/category/rushing/2023/REG/all/rushingyards/DESC?aftercursor=AAAASwAAAEZAJgAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhNU0lzSWpNeU1EQTFNelE0TFRReE1EZ3RPVFEwTnkwek5Ea3hMV1ZpTnpOaFpEQTNaV1l6TUNJc0lqSXdNak1pWFgwPQ==
Scraping next page: https://www.nfl.com/stats/player-stats/category/rushing/2023/REG/all/rushingyards/DESC?aftercursor=AAAAZAAAAGBAFAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STFJaXdpTXpJd01EVTBOREV0T

## Turn to DataFrame

### Passing

#### Page 1

In [7]:
# Request raw html
passing1 = requests.get("https://www.nfl.com/stats/player-stats/category/passing/2023/reg/all/passingyards/desc")

# Create a BeautifulSoup object called `soup` to traverse the combined HTML
soup = BeautifulSoup(passing1.content, "html.parser")

# Inspect HTML code
#print(soup)

In [8]:
# Find all elements with the class 'd3-o-player-stats--detailed'
stats = soup.find_all(attrs={"class": 'd3-o-player-stats--detailed'})

# Initialize an empty list to collect the text from <td> elements and the column names
stat_val = []
stat_col = []

# Loop through each <tr> element to extract and collect the text from <td> elements
for row in stats:
    ### This gets the stat names
    # Find all the <th> elements within the current <tr> element
    header_cells = row.find_all('th')

    # Extract and append the text within each <th> element
    if len(header_cells) > 0:
        for cell in header_cells:
            stat_col.append(cell.get_text(strip=True))


    ### This gets the stats
    # Find all the <td> elements within the current <tr> element
    data_cells = row.find_all('td')

    # Extract and append the text from each <td> element to the list
    if len(data_cells) > 0:
        for cell in data_cells:
            stat_val.append(cell.get_text(strip=True))

# Determine the number of columns in each row
num_columns = len(stat_col) 

# Split the list into rows
rows = [stat_val[i:i + num_columns] for i in range(0, len(stat_val), num_columns)]

# Create a DataFrame
df1 = pd.DataFrame(rows, columns=stat_col)

# Convert all columns except "Player" to numeric
numeric_columns = df1.columns.difference(['Player'])
df1[numeric_columns] = df1[numeric_columns].apply(pd.to_numeric)

# Print the DataFrame
df1

,Player,Pass Yds,Yds/Att,Att,Cmp,Cmp %,TD,INT,Rate,1st,1st%,20+,40+,Lng,Sck,SckY
0,Tua Tagovailoa,466,10.4,45,28,62.2,3,1,110.0,23,51.1,8,1,47,0,0
1,Kirk Cousins,344,7.8,44,33,75.0,2,1,102.8,15,34.1,5,1,42,2,16
2,Matthew Stafford,334,8.8,38,24,63.2,0,0,91.3,17,44.7,6,1,44,0,0
3,Mac Jones,316,5.8,54,35,64.8,3,1,91.3,18,33.3,2,0,32,2,10
4,Derek Carr,305,9.2,33,23,69.7,1,1,96.2,12,36.4,5,2,45,4,23
5,Jared Goff,253,7.2,35,22,62.9,1,0,94.1,11,31.4,4,0,33,1,3
6,Jordan Love,245,9.1,27,15,55.6,3,0,123.2,10,37.0,4,1,51,1,8
7,C.J. Stroud,242,5.5,44,28,63.6,0,0,78.0,13,29.6,1,0,26,5,46
8,Trevor Lawrence,241,7.5,32,24,75.0,2,1,103.8,10,31.2,3,0,29,2,4
9,Kenny Pickett,232,5.0,46,31,67.4,1,2,68.4,9,19.6,2,0,31,5,34


#### Page 2

In [9]:
passing2 = requests.get("https://www.nfl.com/stats/player-stats/category/passing/2023/REG/all/passingyards/DESC?aftercursor=AAAAGQAAABlAYeAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhORE1pTENJek1qQXdOVEExTWkwME5USTRMVFUzTWpNdFpERmlNaTA1Tm1VNU1tVmlZekV5TkRFaUxDSXlNREl6SWwxOQ==")

# Create a BeautifulSoup object called `soup` to traverse the combined HTML
soup = BeautifulSoup(passing2.content, "html.parser")


In [10]:
# Find all elements with the class 'd3-o-player-stats--detailed'
stats = soup.find_all(attrs={"class": 'd3-o-player-stats--detailed'})

# Initialize an empty list to collect the text from <td> elements and the column names
stat_val = []
stat_col = []

# Loop through each <tr> element to extract and collect the text from <td> elements
for row in stats:
    ### This gets the stat names
    # Find all the <th> elements within the current <tr> element
    header_cells = row.find_all('th')

    # Extract and append the text within each <th> element
    if len(header_cells) > 0:
        for cell in header_cells:
            stat_col.append(cell.get_text(strip=True))


    ### This gets the stats
    # Find all the <td> elements within the current <tr> element
    data_cells = row.find_all('td')

    # Extract and append the text from each <td> element to the list
    if len(data_cells) > 0:
        for cell in data_cells:
            stat_val.append(cell.get_text(strip=True))

# Determine the number of columns in each row
num_columns = len(stat_col) 

# Split the list into rows
rows = [stat_val[i:i + num_columns] for i in range(0, len(stat_val), num_columns)]

# Create a DataFrame
df2 = pd.DataFrame(rows, columns=stat_col)

# Convert all columns except "Player" to numeric
numeric_columns = df2.columns.difference(['Player'])
df2[numeric_columns] = df2[numeric_columns].apply(pd.to_numeric)

# Print the DataFrame
df2

,Player,Pass Yds,Yds/Att,Att,Cmp,Cmp %,TD,INT,Rate,1st,1st%,20+,40+,Lng,Sck,SckY
0,Joshua Dobbs,132,4.4,30,21,70.0,0,0,78.8,9,30.0,1,0,31,3,18
1,Desmond Ridder,115,6.4,18,15,83.3,1,0,111.8,6,33.3,1,0,34,4,24
2,Geno Smith,112,4.3,26,16,61.5,1,0,84.1,7,26.9,1,0,28,2,17
3,Daniel Jones,104,3.7,28,15,53.6,0,2,32.4,5,17.9,2,0,24,7,47
4,Joe Burrow,82,2.6,31,14,45.2,0,0,52.2,3,9.7,0,0,12,2,15
5,Tyrod Taylor,6,3.0,2,2,100.0,0,0,79.2,0,0.0,0,0,9,0,0
6,Jake Browning,0,0.0,1,0,0.0,0,0,39.6,0,0.0,0,0,0,0,0
7,Gardner Minshew,0,0.0,2,0,0.0,0,0,39.6,0,0.0,0,0,0,0,0
8,Cooper Rush,0,0.0,1,0,0.0,0,0,39.6,0,0.0,0,0,0,0,0


#### Combine Pages

In [11]:
# Stack them vertically (along rows)
passing = pd.concat([df1, df2])

# Sort by 'Pass Yds' in descending order
passing = passing.sort_values(by='Pass Yds', ascending=False)

# Reindex the dataframe
passing = passing.reset_index(drop=True)
passing

,Player,Pass Yds,Yds/Att,Att,Cmp,Cmp %,TD,INT,Rate,1st,1st%,20+,40+,Lng,Sck,SckY
0,Tua Tagovailoa,466,10.4,45,28,62.2,3,1,110.0,23,51.1,8,1,47,0,0
1,Kirk Cousins,344,7.8,44,33,75.0,2,1,102.8,15,34.1,5,1,42,2,16
2,Matthew Stafford,334,8.8,38,24,63.2,0,0,91.3,17,44.7,6,1,44,0,0
3,Mac Jones,316,5.8,54,35,64.8,3,1,91.3,18,33.3,2,0,32,2,10
4,Derek Carr,305,9.2,33,23,69.7,1,1,96.2,12,36.4,5,2,45,4,23
5,Jared Goff,253,7.2,35,22,62.9,1,0,94.1,11,31.4,4,0,33,1,3
6,Jordan Love,245,9.1,27,15,55.6,3,0,123.2,10,37.0,4,1,51,1,8
7,C.J. Stroud,242,5.5,44,28,63.6,0,0,78.0,13,29.6,1,0,26,5,46
8,Trevor Lawrence,241,7.5,32,24,75.0,2,1,103.8,10,31.2,3,0,29,2,4
9,Kenny Pickett,232,5.0,46,31,67.4,1,2,68.4,9,19.6,2,0,31,5,34


#### Export DataFrame

In [12]:
import os

## Check if directory "data" exists, if not create
# Define the directory path
directory_path = 'data'

# Check if the directory exists
if not os.path.exists(directory_path):
    # Create the directory if it doesn't exist
    os.makedirs(directory_path)
    print(f'Directory "{directory_path}" has been created.')
else:
    print(f'Directory "{directory_path}" already exists.')

Directory "data" already exists.


In [13]:
current_week = 'week1'

## Check if directory "week1" exists, if not create
# Combine the base directory path with the current week
directory_path = os.path.join('data', current_week)

# Check if the directory exists
if not os.path.exists(directory_path):
    # Create the directory if it doesn't exist
    os.makedirs(directory_path)
    print(f'Directory "{directory_path}" has been created.')
else:
    print(f'Directory "{directory_path}" already exists.')

Directory "data\week1" already exists.


In [14]:
# Specify the file path where you want to save the CSV file
csv_file_path = os.path.join('data', current_week, 'passing.csv')

# Export the DataFrame to a CSV file
passing.to_csv(csv_file_path, index=False)  # Set index=False to exclude the index column

print(f'DataFrame has been exported to {csv_file_path}')

DataFrame has been exported to data\week1\passing.csv


### Rushing

#### Page 1

In [15]:
# Request raw html
rushing1 = requests.get("https://www.nfl.com/stats/player-stats/category/rushing/2023/reg/all/rushingyards/desc")

# Create a BeautifulSoup object called `soup` to traverse the combined HTML
soup = BeautifulSoup(rushing1.content, "html.parser")


In [20]:
# Find all elements with the class 'd3-o-player-stats--detailed'
stats = soup.find_all(attrs={"class": 'd3-o-player-stats--detailed'})

# Initialize an empty list to collect the text from <td> elements and the column names
stat_val = []
stat_col = []

# Loop through each <tr> element to extract and collect the text from <td> elements
for row in stats:
    ### This gets the stat names
    # Find all the <th> elements within the current <tr> element
    header_cells = row.find_all('th')

    # Extract and append the text within each <th> element
    if len(header_cells) > 0:
        for cell in header_cells:
            stat_col.append(cell.get_text(strip=True))


    ### This gets the stats
    # Find all the <td> elements within the current <tr> element
    data_cells = row.find_all('td')

    # Extract and append the text from each <td> element to the list
    if len(data_cells) > 0:
        for cell in data_cells:
            stat_val.append(cell.get_text(strip=True))

# Determine the number of columns in each row
num_columns = len(stat_col) 

# Split the list into rows
rows = [stat_val[i:i + num_columns] for i in range(0, len(stat_val), num_columns)]

# Create a DataFrame
df1 = pd.DataFrame(rows, columns=stat_col)

# Convert all columns except "Player" to numeric
numeric_columns = df1.columns.difference(['Player'])
df1[numeric_columns] = df1[numeric_columns].apply(pd.to_numeric)

# Print the DataFrame
df1

,Player,Rush Yds,Att,TD,20+,40+,Lng,Rush 1st,Rush 1st%,Rush FUM
0,Trevor Lawrence,21,7,0,0,0,11,2,28.6,0
1,Roschon Johnson,20,5,1,0,0,11,3,60.0,0
2,Brock Purdy,20,3,0,0,0,17,1,33.3,0
3,C.J. Stroud,20,4,0,0,0,11,1,25.0,0
4,AJ Dillon,19,13,0,0,0,7,0,0.0,0
5,Elijah Moore,19,2,0,0,0,19,1,50.0,0
6,Justin Herbert,18,5,1,0,0,5,4,80.0,0
7,Erik Ezukanma,17,2,0,0,0,12,1,50.0,0
8,Bryce Young,17,3,0,0,0,16,1,33.3,1
9,D'Onta Foreman,16,5,0,0,0,11,2,40.0,0


### Page 2


In [23]:
# Request raw html
rushing2 = requests.get("https://www.nfl.com/stats/player-stats/category/rushing/2023/REG/all/rushingyards/DESC?aftercursor=AAAAGQAAABdARoAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STBOU0lzSWpNeU1EQTFOelE1TFRSak16SXRPREkyT1MwMFlURTVMV1E0TlRGaFl6azVZbUV6WXlJc0lqSXdNak1pWFgwPQ==")

# Create a BeautifulSoup object called `soup` to traverse the combined HTML
soup = BeautifulSoup(rushing2.content, "html.parser")


# Find all elements with the class 'd3-o-player-stats--detailed'
stats = soup.find_all(attrs={"class": 'd3-o-player-stats--detailed'})

# Initialize an empty list to collect the text from <td> elements and the column names
stat_val = []
stat_col = []

# Loop through each <tr> element to extract and collect the text from <td> elements
for row in stats:
    ### This gets the stat names
    # Find all the <th> elements within the current <tr> element
    header_cells = row.find_all('th')

    # Extract and append the text within each <th> element
    if len(header_cells) > 0:
        for cell in header_cells:
            stat_col.append(cell.get_text(strip=True))


    ### This gets the stats
    # Find all the <td> elements within the current <tr> element
    data_cells = row.find_all('td')

    # Extract and append the text from each <td> element to the list
    if len(data_cells) > 0:
        for cell in data_cells:
            stat_val.append(cell.get_text(strip=True))

# Determine the number of columns in each row
num_columns = len(stat_col) 

# Split the list into rows
rows = [stat_val[i:i + num_columns] for i in range(0, len(stat_val), num_columns)]

# Create a DataFrame
df2 = pd.DataFrame(rows, columns=stat_col)

# Convert all columns except "Player" to numeric
numeric_columns = df2.columns.difference(['Player'])
df2[numeric_columns] = df2[numeric_columns].apply(pd.to_numeric)

# Print the DataFrame
df2

,Player,Rush Yds,Att,TD,20+,40+,Lng,Rush 1st,Rush 1st%,Rush FUM
0,Daniel Jones,43,13,0,0,0,11,3,23.1,1
1,Jahmyr Gibbs,42,7,0,0,0,18,2,28.6,0
2,Aaron Jones,41,9,1,0,0,9,3,33.3,0
3,Samaje Perine,41,8,0,0,0,12,3,37.5,0
4,Anthony Richardson,40,10,1,0,0,12,3,30.0,0
5,Rachaad White,39,17,0,0,0,6,1,5.9,0
6,Lamar Jackson,38,6,0,0,0,14,3,50.0,1
7,Dameon Pierce,38,11,0,0,0,7,0,0.0,0
8,Jalen Hurts,37,9,0,0,0,14,2,22.2,1
9,Raheem Mostert,37,10,1,0,0,12,3,30.0,0


### Page 3

In [24]:
rushing3 = requests.get("https://www.nfl.com/stats/player-stats/category/rushing/2023/REG/all/rushingyards/DESC?aftercursor=AAAAMgAAADBANgAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlNaUlzSWpNeU1EQTFORFF4TFRVNU5qRXRNRFUxTXkwNU1UZGpMVE5oTlRabE5XRTBZekkyWXlJc0lqSXdNak1pWFgwPQ==")

# Create a BeautifulSoup object called `soup` to traverse the combined HTML
soup = BeautifulSoup(rushing3.content, "html.parser")


In [25]:
# Find all elements with the class 'd3-o-player-stats--detailed'
stats = soup.find_all(attrs={"class": 'd3-o-player-stats--detailed'})

# Initialize an empty list to collect the text from <td> elements and the column names
stat_val = []
stat_col = []

# Loop through each <tr> element to extract and collect the text from <td> elements
for row in stats:
    ### This gets the stat names
    # Find all the <th> elements within the current <tr> element
    header_cells = row.find_all('th')

    # Extract and append the text within each <th> element
    if len(header_cells) > 0:
        for cell in header_cells:
            stat_col.append(cell.get_text(strip=True))


    ### This gets the stats
    # Find all the <td> elements within the current <tr> element
    data_cells = row.find_all('td')

    # Extract and append the text from each <td> element to the list
    if len(data_cells) > 0:
        for cell in data_cells:
            stat_val.append(cell.get_text(strip=True))

# Determine the number of columns in each row
num_columns = len(stat_col) 

# Split the list into rows
rows = [stat_val[i:i + num_columns] for i in range(0, len(stat_val), num_columns)]

# Create a DataFrame
df3 = pd.DataFrame(rows, columns=stat_col)

# Convert all columns except "Player" to numeric
numeric_columns = df3.columns.difference(['Player'])
df3[numeric_columns] = df3[numeric_columns].apply(pd.to_numeric)

# Print the DataFrame
df3

,Player,Rush Yds,Att,TD,20+,40+,Lng,Rush 1st,Rush 1st%,Rush FUM
0,Trevor Lawrence,21,7,0,0,0,11,2,28.6,0
1,Roschon Johnson,20,5,1,0,0,11,3,60.0,0
2,Brock Purdy,20,3,0,0,0,17,1,33.3,0
3,C.J. Stroud,20,4,0,0,0,11,1,25.0,0
4,AJ Dillon,19,13,0,0,0,7,0,0.0,0
5,Elijah Moore,19,2,0,0,0,19,1,50.0,0
6,Justin Herbert,18,5,1,0,0,5,4,80.0,0
7,Erik Ezukanma,17,2,0,0,0,12,1,50.0,0
8,Bryce Young,17,3,0,0,0,16,1,33.3,1
9,D'Onta Foreman,16,5,0,0,0,11,2,40.0,0


### Page 4

In [26]:
rushing4 = requests.get("https://www.nfl.com/stats/player-stats/category/rushing/2023/REG/all/rushingyards/DESC?aftercursor=AAAASwAAAEZAJgAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhNU0lzSWpNeU1EQTFNelE0TFRReE1EZ3RPVFEwTnkwek5Ea3hMV1ZpTnpOaFpEQTNaV1l6TUNJc0lqSXdNak1pWFgwPQ==")

# Create a BeautifulSoup object called `soup` to traverse the combined HTML
soup = BeautifulSoup(rushing4.content, "html.parser")


In [27]:
# Find all elements with the class 'd3-o-player-stats--detailed'
stats = soup.find_all(attrs={"class": 'd3-o-player-stats--detailed'})

# Initialize an empty list to collect the text from <td> elements and the column names
stat_val = []
stat_col = []

# Loop through each <tr> element to extract and collect the text from <td> elements
for row in stats:
    ### This gets the stat names
    # Find all the <th> elements within the current <tr> element
    header_cells = row.find_all('th')

    # Extract and append the text within each <th> element
    if len(header_cells) > 0:
        for cell in header_cells:
            stat_col.append(cell.get_text(strip=True))


    ### This gets the stats
    # Find all the <td> elements within the current <tr> element
    data_cells = row.find_all('td')

    # Extract and append the text from each <td> element to the list
    if len(data_cells) > 0:
        for cell in data_cells:
            stat_val.append(cell.get_text(strip=True))

# Determine the number of columns in each row
num_columns = len(stat_col) 

# Split the list into rows
rows = [stat_val[i:i + num_columns] for i in range(0, len(stat_val), num_columns)]

# Create a DataFrame
df4 = pd.DataFrame(rows, columns=stat_col)

# Convert all columns except "Player" to numeric
numeric_columns = df4.columns.difference(['Player'])
df4[numeric_columns] = df4[numeric_columns].apply(pd.to_numeric)

# Print the DataFrame
df4

,Player,Rush Yds,Att,TD,20+,40+,Lng,Rush 1st,Rush 1st%,Rush FUM
0,Matthew Stafford,11,3,0,0,0,9,1,33.3,0
1,Jake Funk,10,2,0,0,0,7,0,0.0,0
2,Elijah Mitchell,10,5,0,0,0,4,0,0.0,0
3,Matt Breida,9,2,0,0,0,6,1,50.0,0
4,Treylon Burks,9,1,0,0,0,9,0,0.0,0
5,Zay Flowers,9,2,0,0,0,6,0,0.0,0
6,Antonio Gibson,9,3,0,0,0,3,0,0.0,1
7,Justice Hill,9,8,2,0,0,4,2,25.0,0
8,Chase Edmonds,8,2,0,0,0,5,1,50.0,0
9,Deebo Samuel,8,2,0,0,0,5,0,0.0,0
